# Clase 1.5 - Interfaces y Abstract Base Classes (ABC)

**Unidad:** 1 - POO Avanzada con Python  
**Conexión al Proyecto TaskFlow:** Diseñaremos interfaces para los repositorios del sistema, asegurando que todas las implementaciones sigan el mismo contrato.  

## 📋 Objetivos de Aprendizaje

Al finalizar esta clase, serás capaz de:
- [ ] Comprender el propósito de las interfaces
- [ ] Crear Abstract Base Classes con `ABC`
- [ ] Usar `@abstractmethod` para definir métodos abstractos
- [ ] Implementar interfaces concretas
- [ ] Aplicar Protocolos para type checking estructural

## 💻 Conexión con TaskFlow

En el sistema **TaskFlow**, usaremos interfaces para:
- Definir el contrato de los Repositories (CRUD)
- Asegurar que todos los repos (Usuario, Proyecto, Tarea) tengan los mismos métodos
- Facilitar el cambio entre diferentes implementaciones (en memoria, archivos, BD)
- Permitir testing con mocks y stubs

In [ ]:
# Importaciones
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Optional, TypeVar, Generic
from enum import Enum

---

## 1. 📋 ¿Qué es una Interfaz?

**📐 Definición:** Una **interfaz** es un "contrato" que define qué métodos debe tener una clase, SIN especificar CÓMO se implementan.

**📋 Propósito:**
- Define un contrato que las clases deben cumplir
- Permite polimorfismo: diferentes implementaciones, misma interfaz
- Facilita el testing y la inyección de dependencias

**📈 Analogía:**
- **Interfaz:** El control remoto de un TV (botones: power, volumen, canal)
- **Implementaciones:** Samsung, LG, Sony (diferentes marcas, mismos botones)

**⚠️ En Python:** No existe la palabra clave `interface` (como en Java o C#). En su lugar usamos:
- **ABC (Abstract Base Classes)** para interfaces formales
- **Protocolos** para type checking estructural (duck typing estático)

---

## 2. 🤗 Abstract Base Classes (ABC)

**📐 Definición:** Una **ABC** es una clase que contiene uno o más métodos abstractos. No se puede instanciar directamente; solo se usa como base para otras clases.

**📋 Características:**
- Se crea heredando de `ABC`
- Usa `@abstractmethod` para marcar métodos abstractos
- Python impide instanciar una ABC directamente

**📋 Sintaxis:**
```python
from abc import ABC, abstractmethod

class MiInterfaz(ABC):
    @abstractmethod
    def metodo_obligatorio(self):
        pass  # Sin implementación
```

In [ ]:
# Ejemplo 1: Interfaz Repository para TaskFlow

T = TypeVar('T')  # Tipo genérico

class Repository(ABC, Generic[T]):
    """
    Interfaz base para todos los repositorios de TaskFlow.
    Define el contrato CRUD que todas las implementaciones deben seguir.
    
    Esta es una clase abstracta: no se puede instanciar directamente.
    """
    
    @abstractmethod
    def save(self, entity: T) -> T:
        """
        Guarda una entidad.
        
        Args:
            entity: La entidad a guardar
        
        Returns:
            La entidad guardada (con ID asignado si es nueva)
        """
        pass
    
    @abstractmethod
    def find_by_id(self, entity_id: int) -> Optional[T]:
        """
        Busca una entidad por su ID.
        
        Args:
            entity_id: El ID de la entidad a buscar
        
        Returns:
            La entidad si se encuentra, None en caso contrario
        """
        pass
    
    @abstractmethod
    def find_all(self) -> List[T]:
        """
        Retorna todas las entidades.
        
        Returns:
            Lista de todas las entidades
        """
        pass
    
    @abstractmethod
    def update(self, entity: T) -> T:
        """
        Actualiza una entidad existente.
        
        Args:
            entity: La entidad a actualizar (con ID)
        
        Returns:
            La entidad actualizada
        
        Raises:
            ValueError: Si la entidad no existe
        """
        pass
    
    @abstractmethod
    def delete(self, entity_id: int) -> bool:
        """
        Elimina una entidad por su ID.
        
        Args:
            entity_id: El ID de la entidad a eliminar
        
        Returns:
            True si se eliminó, False si no existía
        """
        pass
    
    @abstractmethod
    def count(self) -> int:
        """
        Retorna la cantidad de entidades.
        
        Returns:
            Cantidad total de entidades
        """
        pass

# Intentar instanciar la ABC genera error
try:
    repo = Repository()
    print("No deberías ver esto")
except TypeError as e:
    print(f"Error esperado: {e}")
    print("\nNo se puede instanciar una clase abstracta directamente.")

**💡 Nota:** Python impide instanciar clases abstractas que tienen métodos abstractos sin implementar.

---

## 3. ✅ Implementación de Interfaces

**📐 Para implementar una interfaz:**
1. Heredar de la ABC
2. Implementar TODOS los métodos abstractos
3. Si falta alguno, Python no permitirá instanciar la clase

**📋 Ventaja:** Garantiza que todas las implementaciones tengan el mismo contrato.

In [ ]:
# Ejemplo 2: Implementación concreta de Repository

@dataclass
class Usuario:
    """Modelo de Usuario."""
    id: Optional[int] = None
    username: str = ""
    email: str = ""


class UsuarioRepositoryInMemory(Repository[Usuario]):
    """
    Implementación en memoria de Repository para Usuarios.
    
    Esta clase DEBE implementar todos los métodos abstractos
    de Repository, o Python no permitirá instanciarla.
    """
    
    def __init__(self):
        self._usuarios: dict[int, Usuario] = {}
        self._next_id: int = 1
    
    def save(self, entity: Usuario) -> Usuario:
        """Guarda un usuario (nuevo o existente)."""
        if entity.id is None:
            # Nuevo usuario: asignar ID
            entity.id = self._next_id
            self._next_id += 1
        
        self._usuarios[entity.id] = entity
        return entity
    
    def find_by_id(self, entity_id: int) -> Optional[Usuario]:
        """Busca un usuario por ID."""
        return self._usuarios.get(entity_id)
    
    def find_all(self) -> List[Usuario]:
        """Retorna todos los usuarios."""
        return list(self._usuarios.values())
    
    def update(self, entity: Usuario) -> Usuario:
        """Actualiza un usuario existente."""
        if entity.id is None or entity.id not in self._usuarios:
            raise ValueError(f"Usuario con ID {entity.id} no existe")
        self._usuarios[entity.id] = entity
        return entity
    
    def delete(self, entity_id: int) -> bool:
        """Elimina un usuario por ID."""
        if entity_id in self._usuarios:
            del self._usuarios[entity_id]
            return True
        return False
    
    def count(self) -> int:
        """Retorna la cantidad de usuarios."""
        return len(self._usuarios)
    
    # Métodos adicionales específicos de UsuarioRepository
    def find_by_username(self, username: str) -> Optional[Usuario]:
        """Busca un usuario por nombre de usuario."""
        for usuario in self._usuarios.values():
            if usuario.username == username:
                return usuario
        return None

# Probar la implementación
repo = UsuarioRepositoryInMemory()

# Crear usuarios
u1 = repo.save(Usuario(username="jdoe", email="john@example.com"))
u2 = repo.save(Usuario(username="asmith", email="alice@example.com"))
u3 = repo.save(Usuario(username="bjones", email="bob@example.com"))

print("=== USUARIOS CREADOS ===")
print(f"Total: {repo.count()}")
for u in repo.find_all():
    print(f"  [{u.id}] {u.username} - {u.email}")

# Buscar por ID
print("\n=== BUSCAR POR ID ===")
encontrado = repo.find_by_id(2)
print(f"ID 2: {encontrado.username if encontrado else 'No encontrado'}")

# Buscar por username (método específico)
print("\n=== BUSCAR POR USERNAME ===")
encontrado = repo.find_by_username("jdoe")
print(f"Username 'jdoe': {encontrado.email if encontrado else 'No encontrado'}")

# Actualizar
print("\n=== ACTUALIZAR ===")
u1.email = "john.doe@example.com"
repo.update(u1)
print(f"Usuario {u1.username} actualizado: {repo.find_by_id(1).email}")

# Eliminar
print("\n=== ELIMINAR ===")
eliminado = repo.delete(3)
print(f"Usuario 3 eliminado: {eliminado}")
print(f"Total actual: {repo.count()}")

**💡 Ventaja:** Podemos cambiar `UsuarioRepositoryInMemory` por `UsuarioRepositoryBD` sin romper el código que usa el repositorio.

---

## 4. 📜 Interfaces en Python vs Java/C#

| Aspecto | Java/C# | Python |
|---------|---------|--------|
| **Palabra clave** | `interface` | `ABC` (clase abstracta) |
| **Implementación** | `implements Interfaz` | Herencia: `class(Interfaz)` |
| **Métodos** | Solo firmas | Firmas + implementación default |
| **Verificación** | En compilación | En instanciación |
| **Herencia múltiple** | No permitida | Permitida (múltiples ABCs) |

In [ ]:
# Ejemplo 3: Implementación faltante de método abstracto

class RepositoryIncompleto(Repository[Usuario]):
    """
    Implementación INCOMPLETA de Repository.
    Falta implementar 'count', por lo que no se puede instanciar.
    """
    
    def __init__(self):
        self._usuarios: dict[int, Usuario] = {}
    
    def save(self, entity: Usuario) -> Usuario:
        if entity.id is None:
            entity.id = 1
        self._usuarios[entity.id] = entity
        return entity
    
    def find_by_id(self, entity_id: int) -> Optional[Usuario]:
        return self._usuarios.get(entity_id)
    
    def find_all(self) -> List[Usuario]:
        return list(self._usuarios.values())
    
    def update(self, entity: Usuario) -> Usuario:
        self._usuarios[entity.id] = entity
        return entity
    
    def delete(self, entity_id: int) -> bool:
        if entity_id in self._usuarios:
            del self._usuarios[entity_id]
            return True
        return False
    
    # FALTA implementar count()!

# Intentar instanciar
try:
    repo_incompleto = RepositoryIncompleto()
    print("No deberías ver esto")
except TypeError as e:
    print(f"Error esperado: {e}")
    print("\nPython no permite instanciar si faltan métodos abstractos.")

**💡 Protección:** Python garantiza en tiempo de ejecución que todas las clases derivadas implementen los métodos abstractos.

---

## 5. 🔰 Protocolos: Type Checking Estructural

**📐 Definición:** Un **Protocolo** define una interfaz estructural. En lugar de verificar la herencia explícita, verifica que un objeto tenga los métodos/atributos necesarios.

**📋 Diferencia clave:**
- **ABC:** Verificación nominal ("es de tipo X")
- **Protocol:** Verificación estructural ("tiene métodos X, Y, Z")

**📋 Ventaja:** Mayor flexibilidad, se alinea con el duck typing de Python.

In [ ]:
# Ejemplo 4: Protocol vs ABC

from typing import Protocol

# Definir un Protocol (interfaz estructural)
class Guardable(Protocol):
    """
    Protocolo para objetos que pueden ser guardados.
    Cualquier clase con un método save() cumple este protocolo.
    """
    def save(self) -> None:
        """Guarda el objeto."""
        ...

# Clases que cumplen el protocolo (sin heredar explícitamente)
@dataclass
class Proyecto:
    """Proyecto que puede ser guardado."""
    nombre: str
    descripcion: str = ""
    
    def save(self) -> None:
        print(f"Guardando proyecto: {self.nombre}")

@dataclass
class Tarea:
    """Tarea que puede ser guardada."""
    titulo: str
    
    def save(self) -> None:
        print(f"Guardando tarea: {self.titulo}")

# Función que usa el protocolo
def guardar_entidad(entidad: Guardable) -> None:
    """
    Guarda cualquier entidad que cumpla el protocolo Guardable.
    No verifica el tipo, solo si tiene el método save().
    """
    entidad.save()

# Funciona con cualquier objeto que tenga save()
proyecto = Proyecto(nombre="TaskFlow", descripcion="Sistema de gestión")
tarea = Tarea(titulo="Implementar auth")

print("=== PROTOCOLO EN ACCIÓN ===")
guardar_entidad(proyecto)  # Funciona: Proyecto tiene save()
guardar_entidad(tarea)     # Funciona: Tarea tiene save()

# Type checkers (mypy) verificarán que solo pasemos objetos con save()
print("\nLos type checkers verificarán que el objeto tenga save().")

**💡 Nota:** Los Protocolos son útiles para type checking estático (mypy) mientras mantienes la flexibilidad de Python.

---

## 6. 📋 Cuándo Usar ABC vs Protocol

| Situación | Usa | Por qué |
|----------|-----|-------- |
| **Requiero estado compartido** | ABC | Las clases pueden heredar atributos y métodos concretos |
| **Quiero verificar en runtime** | ABC | Python impide instanciar si faltan métodos |
| **Quiero type checking estático** | Protocol | Mypy verifica estructura, no herencia |
| **Múltiples implementaciones no relacionadas** | Protocol | No necesitan compartir una clase base |
| **API pública de biblioteca** | ABC | Más explícito y documentado |

In [ ]:
# Ejemplo 5: Sistema completo de repositorios para TaskFlow

# === MODELOS ===
@dataclass
class Usuario:
    id: Optional[int] = None
    username: str = ""
    email: str = ""

@dataclass
class Proyecto:
    id: Optional[int] = None
    nombre: str = ""
    descripcion: str = ""

@dataclass
class Tarea:
    id: Optional[int] = None
    titulo: str = ""
    proyecto_id: Optional[int] = None

# === INTERFAZ REPOSITORY BASE ===
T = TypeVar('T')

class Repository(ABC, Generic[T]):
    """Interfaz base para repositorios."""
    
    @abstractmethod
    def save(self, entity: T) -> T:
        pass
    
    @abstractmethod
    def find_by_id(self, entity_id: int) -> Optional[T]:
        pass
    
    @abstractmethod
    def find_all(self) -> List[T]:
        pass
    
    @abstractmethod
    def delete(self, entity_id: int) -> bool:
        pass
    
    @abstractmethod
    def count(self) -> int:
        pass

# === IMPLEMENTACIONES ===
class UsuarioRepository(Repository[Usuario]):
    """Repositorio de usuarios en memoria."""
    
    def __init__(self):
        self._data: dict[int, Usuario] = {}
        self._next_id = 1
    
    def save(self, entity: Usuario) -> Usuario:
        if entity.id is None:
            entity.id = self._next_id
            self._next_id += 1
        self._data[entity.id] = entity
        return entity
    
    def find_by_id(self, entity_id: int) -> Optional[Usuario]:
        return self._data.get(entity_id)
    
    def find_all(self) -> List[Usuario]:
        return list(self._data.values())
    
    def delete(self, entity_id: int) -> bool:
        if entity_id in self._data:
            del self._data[entity_id]
            return True
        return False
    
    def count(self) -> int:
        return len(self._data)

class ProyectoRepository(Repository[Proyecto]):
    """Repositorio de proyectos en memoria."""
    
    def __init__(self):
        self._data: dict[int, Proyecto] = {}
        self._next_id = 1
    
    def save(self, entity: Proyecto) -> Proyecto:
        if entity.id is None:
            entity.id = self._next_id
            self._next_id += 1
        self._data[entity.id] = entity
        return entity
    
    def find_by_id(self, entity_id: int) -> Optional[Proyecto]:
        return self._data.get(entity_id)
    
    def find_all(self) -> List[Proyecto]:
        return list(self._data.values())
    
    def delete(self, entity_id: int) -> bool:
        if entity_id in self._data:
            del self._data[entity_id]
            return True
        return False
    
    def count(self) -> int:
        return len(self._data)

# === USAR LOS REPOSITORIOS ===
usuario_repo = UsuarioRepository()
proyecto_repo = ProyectoRepository()

# Crear usuarios
usuario_repo.save(Usuario(username="admin", email="admin@taskflow.com"))
usuario_repo.save(Usuario(username="jdoe", email="john@example.com"))

# Crear proyectos
proyecto_repo.save(Proyecto(nombre="TaskFlow", descripcion="Sistema de gestión"))
proyecto_repo.save(Proyecto(nombre="App Móvil", descripcion="Aplicación Android"))

print("=== REPOSITORIOS TASKFLOW ===")
print(f"Usuarios: {usuario_repo.count()}")
print(f"Proyectos: {proyecto_repo.count()}")

print("\nUsuarios:")
for u in usuario_repo.find_all():
    print(f"  [{u.id}] {u.username}")

print("\nProyectos:")
for p in proyecto_repo.find_all():
    print(f"  [{p.id}] {p.nombre}")

**💡 Ventaja del patrón Repository:** Si luego queremos cambiar a PostgreSQL, solo creamos una nueva implementación de `Repository` sin cambiar el código que lo usa.

---

## 📝 Ejercicio Práctico: Repository de Tareas

Completa la implementación de `TareaRepository` que:

1. Implementa todos los métodos abstractos de `Repository[Tarea]`
2. Usa un diccionario interno para almacenamiento
3. Genera IDs automáticamente
4. Incluye un método adicional `find_by_proyecto(proyecto_id)`
5. Incluye un método adicional `find_pendientes()` que retorne tareas sin proyecto

In [ ]:
# Tu solución aquí

class TareaRepository(Repository[Tarea]):
    """
    Repositorio de tareas en memoria.
    Debe implementar todos los métodos de Repository[Tarea].
    """
    
    def __init__(self):
        # TODO: Inicializar almacenamiento interno
        pass
    
    def save(self, entity: Tarea) -> Tarea:
        # TODO: Implementar save
        pass
    
    def find_by_id(self, entity_id: int) -> Optional[Tarea]:
        # TODO: Implementar find_by_id
        pass
    
    def find_all(self) -> List[Tarea]:
        # TODO: Implementar find_all
        pass
    
    def delete(self, entity_id: int) -> bool:
        # TODO: Implementar delete
        pass
    
    def count(self) -> int:
        # TODO: Implementar count
        pass
    
    def find_by_proyecto(self, proyecto_id: int) -> List[Tarea]:
        """
        Busca tareas por proyecto.
        
        Args:
            proyecto_id: ID del proyecto
        
        Returns:
            Lista de tareas del proyecto
        """
        # TODO: Implementar
        pass
    
    def find_pendientes(self) -> List[Tarea]:
        """
        Busca tareas sin proyecto asignado.
        
        Returns:
            Lista de tareas pendientes de asignar
        """
        # TODO: Implementar
        pass

### ✅ Validación Automática

In [ ]:
# Tests automáticos

def test_tarea_repository():
    """Valida TareaRepository."""
    
    repo = TareaRepository()
    
    # Test 1: Crear tareas
    t1 = repo.save(Tarea(titulo="Tarea 1", proyecto_id=1))
    t2 = repo.save(Tarea(titulo="Tarea 2", proyecto_id=1))
    t3 = repo.save(Tarea(titulo="Tarea 3", proyecto_id=2))
    t4 = repo.save(Tarea(titulo="Tarea sin proyecto"))  # Sin proyecto
    
    assert t1.id == 1, "La primera tarea debe tener ID 1"
    assert t2.id == 2, "La segunda tarea debe tener ID 2"
    print("✅ Test 1: IDs generados correctamente")
    
    # Test 2: Contar
    assert repo.count() == 4
    print("✅ Test 2: count() funciona")
    
    # Test 3: find_by_id
    encontrado = repo.find_by_id(2)
    assert encontrado is not None
    assert encontrado.titulo == "Tarea 2"
    print("✅ Test 3: find_by_id() funciona")
    
    # Test 4: find_all
    todas = repo.find_all()
    assert len(todas) == 4
    print("✅ Test 4: find_all() funciona")
    
    # Test 5: find_by_proyecto
    tareas_p1 = repo.find_by_proyecto(1)
    assert len(tareas_p1) == 2
    tareas_p2 = repo.find_by_proyecto(2)
    assert len(tareas_p2) == 1
    print("✅ Test 5: find_by_proyecto() funciona")
    
    # Test 6: find_pendientes
    pendientes = repo.find_pendientes()
    assert len(pendientes) == 1
    assert pendientes[0].titulo == "Tarea sin proyecto"
    print("✅ Test 6: find_pendientes() funciona")
    
    # Test 7: delete
    eliminado = repo.delete(2)
    assert eliminado is True
    assert repo.count() == 3
    assert repo.find_by_id(2) is None
    print("✅ Test 7: delete() funciona")
    
    # Test 8: delete de inexistente
    eliminado = repo.delete(999)
    assert eliminado is False
    print("✅ Test 8: delete() de inexistente retorna False")
    
    print("\n🚀 ¡Todos los tests pasaron!")

# Ejecutar tests
try:
    test_tarea_repository()
except (AssertionError, AttributeError, TypeError) as e:
    print(f"\n❌ {e}")
    print("\nRevisa tu implementación.")

### 📋 Solución del Ejercicio

In [ ]:
# Solución completa

class TareaRepository(Repository[Tarea]):
    """Repositorio de tareas en memoria."""
    
    def __init__(self):
        self._data: dict[int, Tarea] = {}
        self._next_id = 1
    
    def save(self, entity: Tarea) -> Tarea:
        if entity.id is None:
            entity.id = self._next_id
            self._next_id += 1
        self._data[entity.id] = entity
        return entity
    
    def find_by_id(self, entity_id: int) -> Optional[Tarea]:
        return self._data.get(entity_id)
    
    def find_all(self) -> List[Tarea]:
        return list(self._data.values())
    
    def delete(self, entity_id: int) -> bool:
        if entity_id in self._data:
            del self._data[entity_id]
            return True
        return False
    
    def count(self) -> int:
        return len(self._data)
    
    def find_by_proyecto(self, proyecto_id: int) -> List[Tarea]:
        """Busca tareas por proyecto."""
        return [t for t in self._data.values() if t.proyecto_id == proyecto_id]
    
    def find_pendientes(self) -> List[Tarea]:
        """Busca tareas sin proyecto asignado."""
        return [t for t in self._data.values() if t.proyecto_id is None]

# Ejecutar tests de nuevo
test_tarea_repository()

---

## 📈 Diagrama: Patrón Repository en TaskFlow

```mermaid
graph TB
    A[Repository ABC Generic] --> B[UsuarioRepository]
    A --> C[ProyectoRepository]
    A --> D[TareaRepository]
    
    B --> E[UsuarioRepoInMemory]
    B --> F[UsuarioRepoBD]
    B --> G[UsuarioRepoArchivos]
    
    H[Service Layer] --> B
    H --> C
    H --> D
    
    style A fill:#e1f5ff
    style E fill:#c8e6c9
    style F fill:#fff9c4
    style G fill:#ffccbc
    style H fill:#d1c4e9
```

---

## 📝 Resumen de la Clase

### 📋 Conceptos Clave

| Concepto | Propósito | Ejemplo |
|----------|-----------|----------|
| **Interfaz** | Contrato de métodos | `Repository` con save, find, delete |
| **ABC** | Clase abstracta base | `class Repository(ABC)` |
| **@abstractmethod** | Método sin implementación | `def save(self): pass` |
| **Protocol** | Type checking estructural | `class Guardable(Protocol)` |
| **Generic[T]** | Tipo genérico | `Repository[Usuario]` |

### 📋 Patrón Repository

**✅ Ventajas:**
- Desacopla la lógica de negocio del almacenamiento
- Facilita cambiar la implementación (en memoria → BD)
- Hace el código testeable (mocks fáciles)
- Centraliza la lógica de acceso a datos

### 📋 ABC vs Protocol: Cuándo usar cada uno

| Situación | Usa |
|----------|------|
| API pública de biblioteca | ABC |
| Verificación en runtime | ABC |
| Type checking con mypy | Protocol |
| Duck typing estático | Protocol |

### 🤝 Conexión con TaskFlow

Hemos diseñado:
- Interfaz `Repository[T]` genérica con CRUD completo
- Implementaciones en memoria para Usuario, Proyecto, Tarea
- Contrato garantizado por ABC (no se puede olvidar métodos)
- Facilidad para cambiar a implementaciones de BD en el futuro

**📈 Próxima clase:** Diseño de Modelos y DTOs

Aprenderemos a:
- Usar `@dataclass` avanzado con validaciones
- Crear modelos con Pydantic
- Implementar DTOs para transferencia de datos
- Validar datos de entrada con Pydantic